[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nmickevicius/mcwBiophysicsMriCourse/blob/main/reconstructionDemo.ipynb)

# Image Reconstruction Demo 

3D balanced steady-state free precession (bSSFP) data were acquired in a phantom. The data were acquired with an acquisition matrix of 256x256x32 with a TR of 4.5 ms and flip angle of 30 degrees. Following an inverse Fourier transform along the "slice" dimension, a single slice was extracted and saved for this demo. 

## Imports and Data Download

In [ ]:
import numpy as np 
import h5py 
import matplotlib.pyplot as plt 

# download the .h5 file and load it as a complex-valued numpy array
!wget --no-check-certificate --content-disposition https://github.com/nmickevicius/mcwBiophysicsMriCourse/raw/main/data/bssfpKSpace.h5
with h5py.File('bssfpKSpace.h5','r') as F:
    data = np.array(F['data'], dtype=np.complex64)
    csm = np.array(F['csm'], dtype=np.complex64)

# Support Functions 

In [ ]:
def fft1d(inp,dim):
    return np.fft.fftshift(np.fft.fft(np.fft.fftshift(inp,dim),axis=dim),dim)

def ifft1d(inp,dim):
    return np.fft.ifftshift(np.fft.ifft(np.fft.ifftshift(inp,dim),axis=dim),dim)

def fft2d(inp):
    return fft1d(fft1d(inp,0),1)

def ifft2d(inp):
    return ifft1d(ifft1d(inp,0),1)

## Simple Inverse FFT Reconstruction

In [ ]:
# inverse FFT to get a 2D image for each coil 
rec = ifft2d(data)

# root sum-of-squares coil combination 
rss = np.sqrt(np.sum(np.abs(rec * np.conj(rec)),axis=-1))

# complex coil combination using sensitivity maps keeps the image phase! 
ccc = np.sum( rec * np.conj(csm), axis=-1)

# display RSS magnitude and phase
fig, axs = plt.subplots(1, 2, figsize=(9, 3))
magim = axs[0].imshow(np.abs(rss), cmap='gray')
plt.colorbar(magim)
phaim = axs[1].imshow(np.angle(rss), vmin=-np.pi, vmax=np.pi)
plt.colorbar(phaim)
fig.suptitle('Root Sum-of-Squares Coil Combination')

# display CCC magnitude and phase
fig, axs = plt.subplots(1, 2, figsize=(9, 3))
magim = axs[0].imshow(np.abs(ccc), cmap='gray')
plt.colorbar(magim)
phaim = axs[1].imshow(np.angle(ccc), vmin=-np.pi, vmax=np.pi)
plt.colorbar(phaim)
fig.suptitle('Complex Coil Combination')

## Undersampling Data by Factor of Two

What happens if we skipped the acquisition of every other line in k-space? 

In [ ]:
usdata = np.zeros_like(data)
usdata[:,::2,:] = data[:,::2,:]

# inverse FFT to get a 2D image for each coil 
rec = ifft2d(usdata)

# root sum-of-squares coil combination 
rss = np.sqrt(np.sum(np.abs(rec * np.conj(rec)),axis=-1))

fig = plt.figure() 
fig.set_figwidth(10)
fig.set_figheight(10)
plt.imshow(np.log10(np.abs(usdata[:,:,0])))
plt.title('2x Accelerated k-Space')
plt.show()

fig = plt.figure() 
fig.set_figwidth(10)
fig.set_figheight(10)
plt.imshow(np.abs(rss), cmap='gray')
plt.title('1x Accelerated Reconstruction')
plt.show()


# Now With a Factor of Four

In [ ]:
usdata = np.zeros_like(data)
usdata[:,::4,:] = data[:,::4,:]

# inverse FFT to get a 2D image for each coil 
rec = ifft2d(usdata)

# root sum-of-squares coil combination 
rss = np.sqrt(np.sum(np.abs(rec * np.conj(rec)),axis=-1))

fig = plt.figure() 
fig.set_figwidth(10)
fig.set_figheight(10)
plt.imshow(np.log10(np.abs(usdata[:,:,0])))
plt.title('2x Accelerated k-Space')
plt.show()

fig = plt.figure() 
fig.set_figwidth(10)
fig.set_figheight(10)
plt.imshow(np.abs(rss), cmap='gray')
plt.title('1x Accelerated Reconstruction')
plt.show()